<a href="https://colab.research.google.com/github/tafartech/Personal-Fitness-Tracker/blob/main/Personal_Fitness_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile

**GPU set up**

In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU is available. Using {torch.cuda.get_device_name(0)}.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

# Display device information
print(f"Current device: {device}")

GPU is available. Using Tesla T4.
Current device: cuda


# Download the HMDB51 dataset.
1. Extract the dataset.
2. Preprocess the data: Convert videos to frames, resize them, and organize them for annotation.

In [ ]:
import os
import requests
import rarfile
import cv2

# Step 1: Download HMDB51 dataset
dataset_url = "https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar"
dataset_path = "hmdb51_org.rar"

def download_dataset(url, path):
    response = requests.get(url, stream=True)
    with open(path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print(f"Downloaded {path}")

download_dataset(dataset_url, dataset_path)

# Step 2: Extract the dataset
extracted_folder = "hmdb51"

def extract_dataset(path, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with rarfile.RarFile(path) as rar_ref:
        rar_ref.extractall(folder)
    print(f"Extracted to {folder}")

extract_dataset(dataset_path, extracted_folder)

# Step 3: Preprocess the data
def video_to_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frame_filename = os.path.join(output_folder, f"frame_{count:05d}.jpg")
        cv2.imwrite(frame_filename, frame)
        count += 1
    cap.release()
    print(f"Converted {video_path} to frames in {output_folder}")

# Convert all videos in the dataset
for root, dirs, files in os.walk(extracted_folder):
    for file in files:
        if file.endswith(".avi"):
            video_path = os.path.join(root, file)
            output_folder = os.path.join(root, file.split('.')[0])
            video_to_frames(video_path, output_folder)

Downloaded hmdb51_org.rar
Extracted to hmdb51


**Install LabelImg:**
I need to install LabelImg in the environment:

In [ ]:
!pip install labelImg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 11.1 MB/s eta 0:00:00
  Created wheel for labelImg: filename=labelImg-1.8.6-py2.py3-none-any.whl size=261519 sha256=b3662faafe4e4be59310cf4fb040cd278275a2f4c381c741b40287875d46a197
  Stored in directory: /root/.cache/pip/wheels/d9/f0/1e/74c509495458cad13a0fda23fe605e643177625cf2b5c17b34
Successfully built labelImg


## Load the frames from the dataset
Load the frames and launch LabelImg for annotation:

In [ ]:
import os
import subprocess

# Path to the frames directory
frames_dir = "hmdb51"

# Install LabelImg if not already installed
!pip install labelImg

# Launch LabelImg for annotation
subprocess.run(["labelImg", frames_dir])

CompletedProcess(args=['labelImg', 'hmdb51'], returncode=-6)

### Organize the Dataset
I will start by organizing the frames into training and validation directories. Here is a script to move the frames into the appropriate directories:

In [ ]:
import os
import rarfile

# Define paths
dataset_path = "/content/hmdb51_org.rar"
extracted_folder = "/content/hmdb51"

# Function to extract RAR file
def extract_dataset(rar_path, folder):
    with rarfile.RarFile(rar_path) as rf:
        rf.extractall(folder)
    print(f"Extracted to {folder}")

# Extract the dataset
extract_dataset(dataset_path, extracted_folder)

# Verify the extracted structure
extracted_subfolders = os.listdir(extracted_folder)
print(f"Subfolders in extracted dataset: {extracted_subfolders}")

Extracted to /content/hmdb51
Subfolders in extracted dataset: ['somersault.rar', 'dive.rar', 'shoot_bow.rar', 'push.rar', 'smoke.rar', 'punch.rar', 'drink.rar', 'shoot_ball.rar', 'sword_exercise.rar', 'pullup.rar', 'run.rar', 'pour.rar', 'talk.rar', 'turn.rar', 'wave.rar', 'sword.rar', 'handstand.rar', 'fall_floor.rar', 'hug.rar', 'eat.rar', 'jump.rar', 'smile.rar', 'climb_stairs.rar', 'hit.rar', 'shoot_gun.rar', 'walk.rar', 'swing_baseball.rar', 'clap.rar', 'cartwheel.rar', 'chew.rar', 'brush_hair.rar', 'ride_horse.rar', 'laugh.rar', 'golf.rar', 'situp.rar', 'pushup.rar', 'kick.rar', 'climb.rar', 'draw_sword.rar', 'dribble.rar', 'catch.rar', 'pick.rar', 'flic_flac.rar', 'stand.rar', 'ride_bike.rar', 'throw.rar', 'shake_hands.rar', 'fencing.rar', 'kick_ball.rar', 'kiss.rar', 'sit.rar']


### Organize the Dataset into Train and Validation Sets

In [ ]:
import os
import rarfile
import shutil
from tqdm import tqdm

# Define paths
dataset_base_path = "/content/yolo_hmdb51"
extracted_folder = "/content/hmdb51"

# Create the directory structure
os.makedirs(os.path.join(dataset_base_path, "images/train"), exist_ok=True)
os.makedirs(os.path.join(dataset_base_path, "images/val"), exist_ok=True)
os.makedirs(os.path.join(dataset_base_path, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(dataset_base_path, "labels/val"), exist_ok=True)

# Function to extract images from activity RAR files
def extract_activity_files(activity_rar_path, destination_folder):
    with rarfile.RarFile(activity_rar_path) as rf:
        rf.extractall(destination_folder)

# Extract and organize dataset
train_ratio = 0.8

for activity_rar in os.listdir(extracted_folder):
    activity_name = os.path.splitext(activity_rar)[0]
    temp_extract_folder = os.path.join(extracted_folder, activity_name)
    os.makedirs(temp_extract_folder, exist_ok=True)

    # Extract images
    extract_activity_files(os.path.join(extracted_folder, activity_rar), temp_extract_folder)

    # Get all image files
    all_files = [f for f in os.listdir(temp_extract_folder) if f.endswith('.jpg')]
    num_train = int(len(all_files) * train_ratio)

    train_files = all_files[:num_train]
    val_files = all_files[num_train:]

    # Move files to train and val directories
    for file in train_files:
        shutil.move(os.path.join(temp_extract_folder, file), os.path.join(dataset_base_path, "images/train", f"{activity_name}_{file}"))

    for file in val_files:
        shutil.move(os.path.join(temp_extract_folder, file), os.path.join(dataset_base_path, "images/val", f"{activity_name}_{file}"))

    # Remove temporary extraction folder
    shutil.rmtree(temp_extract_folder)

print("Dataset organized into train and validation sets.")

Dataset organized into train and validation sets.


In [ ]:
import os
import shutil
import random

# Function to extract frames from video files
def extract_frames(video_file, dest_folder, frame_rate=1):
    import cv2
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    cap = cv2.VideoCapture(video_file)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_filename = os.path.join(dest_folder, f"{os.path.basename(video_file).split('.')[0]}_frame{count}.jpg")
            cv2.imwrite(frame_filename, frame)
        count += 1
    cap.release()

# Define paths
dataset_dir = '/content/hmdb51'
extracted_folder = '/content/hmdb51_extracted'
dataset_base_path = "/content/yolo_hmdb51"
image_train_path = os.path.join(dataset_base_path, "images/train")
image_val_path = os.path.join(dataset_base_path, "images/val")
label_train_path = os.path.join(dataset_base_path, "labels/train")
label_val_path = os.path.join(dataset_base_path, "labels/val")

# Create directories if they don't exist
os.makedirs(image_train_path, exist_ok=True)
os.makedirs(image_val_path, exist_ok=True)
os.makedirs(label_train_path, exist_ok=True)
os.makedirs(label_val_path, exist_ok=True)

# Check for videos in the extracted dataset
for subfolder in os.listdir(dataset_dir):
    subfolder_path = os.path.join(dataset_dir, subfolder)
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            if file.endswith(".avi"):  # Assuming videos are in .avi format
                video_file_path = os.path.join(subfolder_path, file)
                # Extract frames to a temporary folder
                temp_folder = '/content/temp_frames'
                extract_frames(video_file_path, temp_folder)
                # Split frames into train and val
                all_frames = os.listdir(temp_folder)
                random.shuffle(all_frames)
                split_index = int(0.8 * len(all_frames))
                train_frames = all_frames[:split_index]
                val_frames = all_frames[split_index:]
                # Move frames to respective directories
                for frame in train_frames:
                    shutil.move(os.path.join(temp_folder, frame), os.path.join(image_train_path, frame))
                for frame in val_frames:
                    shutil.move(os.path.join(temp_folder, frame), os.path.join(image_val_path, frame))
                # Clean up temp folder
                shutil.rmtree(temp_folder)

print("Frames organized into training and validation sets.")

Frames organized into training and validation sets.


### Convert Annotations
Now, I'll convert the annotations from VOC to YOLO format. Adjust the script to reflect the correct paths:

In [ ]:
import xml.etree.ElementTree as ET
import cv2

# Function to convert VOC to YOLO format
def convert_voc_to_yolo(xml_file, output_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(output_file, 'w') as f:
        for obj in root.findall('object'):
            class_name = obj.find('name').text  # Change this to your class name handling logic
            class_id = 0  # Assuming a single class; update accordingly
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = ((b[0] + b[1]) / 2.0 / img_width, (b[2] + b[3]) / 2.0 / img_height,
                  (b[1] - b[0]) / img_width, (b[3] - b[2]) / img_height)
            f.write(f"{class_id} {' '.join([str(a) for a in bb])}\n")

# Process each file in the dataset
for phase in ['train', 'val']:
    image_dir = f"{output_dir}/images/{phase}"
    label_dir = f"{output_dir}/labels/{phase}"
    for img_file in os.listdir(image_dir):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(image_dir, img_file)
            xml_file = img_path.replace(".jpg", ".xml")  # Adjust path to match annotation location
            yolo_file = os.path.join(label_dir, img_file.replace(".jpg", ".txt"))

            # Read the image to get its dimensions
            img = cv2.imread(img_path)
            height, width, _ = img.shape

            # Convert and save the annotation
            convert_voc_to_yolo(xml_file, yolo_file, width, height)